In [1]:
import glob
import re
import pandas as pd
from os import listdir

bids_folder = '/shares/zne.uzh/mrenke/ds-dnumrisk'
subject_list = [int(f[4:]) for f in listdir(bids_folder) if f[0:3] == 'sub' and len(f) == 6]
all_subjects_set = set(subject_list)
print(*list(all_subjects_set), sep=',')

1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66


### fmriprep

In [2]:
key = 'brain_mask'# 'preproc_bold' #  needed for nprf fitting (usually takes run-1 brain mask)
fns = glob.glob(f'/shares/zne.uzh/mrenke/ds-dnumrisk/derivatives/fmriprep/sub-*/ses-*/func/*space-T1w_desc-{key}.nii.gz')
reg = re.compile(f'.*/sub-(?P<subject>[0-9]+)_ses-1_task-magjudge_run-(?P<run>[0-9]+)_(space-T1w_desc-{key}.nii.gz)') # (?P<session>[0-9]+)

In [3]:
data = []

for fn in fns:
    try:
        data.append(reg.match(fn).groupdict())
        data[-1]['fn'] = fn
    except Exception as e:
        print(f'Problem with {fn}: {e}')

data = pd.DataFrame(data)
data['subject'] = [int(sub) for sub in data['subject']]
df = data.set_index(['subject',  'run']).unstack('run') # 'session',

In [4]:
df[df.isna().any(axis=1)]

fn  \
run                                                      1   
subject                                                      
4        /shares/zne.uzh/mrenke/ds-dnumrisk/derivatives...   
5        /shares/zne.uzh/mrenke/ds-dnumrisk/derivatives...   
47       /shares/zne.uzh/mrenke/ds-dnumrisk/derivatives...   
53       /shares/zne.uzh/mrenke/ds-dnumrisk/derivatives...   
62       /shares/zne.uzh/mrenke/ds-dnumrisk/derivatives...   

                                                            \
run                                                      2   
subject                                                      
4        /shares/zne.uzh/mrenke/ds-dnumrisk/derivatives...   
5        /shares/zne.uzh/mrenke/ds-dnumrisk/derivatives...   
47                                                     NaN   
53                                                     NaN   
62       /shares/zne.uzh/mrenke/ds-dnumrisk/derivatives...   

                                                            \
run                                                      3   
subject                                                      
4        /shares/zne.uzh/mrenke/ds-dnumrisk/derivatives...   
5        /shares/zne.uzh/mrenke/ds-dnumrisk/derivatives...   
47       /shares/zne.uzh/mrenke/ds-dnumrisk/derivatives...   
53       /shares/zne.uzh/mrenke/ds-dnumrisk/derivatives...   
62                                                     NaN   

                                                            \
run                                                      4   
subject                                                      
4        /shares/zne.uzh/mrenke/ds-dnumrisk/derivatives...   
5        /shares/zne.uzh/mrenke/ds-dnumrisk/derivatives...   
47       /shares/zne.uzh/mrenke/ds-dnumrisk/derivatives...   
53       /shares/zne.uzh/mrenke/ds-dnumrisk/derivatives...   
62       /shares/zne.uzh/mrenke/ds-dnumrisk/derivatives...   

                                                            \
run                                                      5   
subject                                                      
4                                                      NaN   
5                                                      NaN   
47       /shares/zne.uzh/mrenke/ds-dnumrisk/derivatives...   
53       /shares/zne.uzh/mrenke/ds-dnumrisk/derivatives...   
62       /shares/zne.uzh/mrenke/ds-dnumrisk/derivatives...   

                                                            
run                                                      6  
subject                                                     
4        /shares/zne.uzh/mrenke/ds-dnumrisk/derivatives...  
5        /shares/zne.uzh/mrenke/ds-dnumrisk/derivatives...  
47       /shares/zne.uzh/mrenke/ds-dnumrisk/derivatives...  
53       /shares/zne.uzh/mrenke/ds-dnumrisk/derivatives...  
62       /shares/zne.uzh/mrenke/ds-dnumrisk/derivatives...

In [5]:
df_subjects_set = set(df.index)
missing_subjects = all_subjects_set - df_subjects_set
missing_subjects

set()

### GLM single denoise


In [2]:
n_stim = 2
fns = glob.glob(f'/shares/zne.uzh/mrenke/ds-dnumrisk/derivatives/glm_stim{n_stim}.denoise/sub-*/ses-*/func/*ses-1_task-magjudge_space-T1w_desc-stims{n_stim}_pe.nii.gz')
reg = re.compile(f'.*/sub-(?P<subject>[0-9]+)_ses-1_task-magjudge_space-T1w_desc-stims{n_stim}_pe.nii.gz') # (?P<session>[0-9]+)

In [3]:
data = []
for fn in fns:
    try:
        data.append(reg.match(fn).groupdict())
        data[-1]['fn'] = fn
    except Exception as e:
        print(f'Problem with {fn}: {e}')

data = pd.DataFrame(data)
data['subject'] = [int(sub) for sub in data['subject']]
df = data.set_index(['subject'])
df.sort_values(by='subject').head()

,fn
subject,
1,/shares/zne.uzh/mrenke/ds-dnumrisk/derivatives...
2,/shares/zne.uzh/mrenke/ds-dnumrisk/derivatives...
3,/shares/zne.uzh/mrenke/ds-dnumrisk/derivatives...
4,/shares/zne.uzh/mrenke/ds-dnumrisk/derivatives...
5,/shares/zne.uzh/mrenke/ds-dnumrisk/derivatives...


In [5]:
all_subjects_set = set(subject_list)
df_subjects_set = set(df.index)

missing_subjects = all_subjects_set - df_subjects_set
print(*list(missing_subjects), sep=',')
#missing_subjects

encoding model (NOT cv)

In [ ]:
n_stim = 2
stim = f'_stim{n_stim}' # '' for 1
fns = glob.glob(f'/shares/zne.uzh/mrenke/ds-dnumrisk/derivatives/encoding_model{stim}.denoise/sub-*/ses-*/func/*desc-r2.optim_space-T1w_pars.nii.gz')
reg = re.compile('.*/sub-(?P<subject>[0-9]+)_ses-1_(desc-r2.optim_space-T1w_pars.nii.gz)') # (?P<session>[0-9]+)

data = []

for fn in fns:
    try:
        data.append(reg.match(fn).groupdict())
        data[-1]['fn'] = fn
    except Exception as e:
        print(f'Problem with {fn}: {e}')

data = pd.DataFrame(data)
data['subject'] = [int(sub) for sub in data['subject']]
df = data.set_index('subject')
df_subjects_set = set(df.index)

missing_subjects = all_subjects_set - df_subjects_set
missing_subjects

In [28]:
print(*missing_subjects, sep=',') # 

3


### encoding model (cv)

In [6]:
n_stim = 2
stim = f'_stim{n_stim}' # '' for 1
fns = glob.glob(f'/shares/zne.uzh/mrenke/ds-dnumrisk/derivatives/encoding_model{stim}.cv.denoise/sub-*/ses-*/func/*desc-r2.optim_space-T1w_pars.nii.gz')
reg = re.compile('.*/sub-(?P<subject>[0-9]+)_ses-1_run-(?P<run>[0-9]+)_(desc-r2.optim_space-T1w_pars.nii.gz)') # (?P<session>[0-9]+)

In [7]:
data = []

for fn in fns:
    try:
        data.append(reg.match(fn).groupdict())
        data[-1]['fn'] = fn
    except Exception as e:
        print(f'Problem with {fn}: {e}')

data = pd.DataFrame(data)
data['subject'] = [int(sub) for sub in data['subject']]
df = data.set_index(['subject',  'run']).unstack('run') # 'session',

In [8]:
df_subjects_set = set(df.index)

missing_subjects = all_subjects_set - df_subjects_set
print(*missing_subjects, sep=',') # 
# log: 'ValueError: File not found: '...._run-1_space-T1w_desc-brain_mask.nii.gz'

3,21


### ips masks

In [10]:
fns = glob.glob('/shares/zne.uzh/mrenke/ds-dnumrisk/derivatives/ips_masks/sub-*/*_space-T1w_desc-NPC_R.nii.gz')
reg = re.compile('.*/sub-(?P<subject>[0-9]+)_(space-T1w_desc-NPC_R.nii.gz)')

In [11]:
data = []
for fn in fns:
    try:
        data.append(reg.match(fn).groupdict())
        data[-1]['fn'] = fn
    except Exception as e:
        print(f'Problem with {fn}: {e}')

data = pd.DataFrame(data)
data['subject'] = [int(sub) for sub in data['subject']]
df = data.set_index(['subject'])
#df.sort_values(by='subject')

df_subjects_set = set(df.index)
missing_subjects = all_subjects_set - df_subjects_set
missing_subjects

set()

### decoding

on sciencecluster2: a few nodes have problems
`CUDA_ERROR_NO_BINARY_FOR_GPU: no kernel image is available for execution on the device` suggests that the CUDA binary (CUBIN) for the GPU is missing or incompatible with the GPU architecture.

check which nodes via 
`sacct -j XXXXXXX --format=JobID,JobName%20,State,Node`

and then specify like
`sbatch --array=52,54,59,60,61,63 submit_decode_vselect.sh --exclude=u20-chaiam0-613,u20-chaiam0-614`


check: https://slurm.schedmd.com/sbatch.html

In [8]:
n_stim = 2
stim = f'_stim{n_stim}' # '' for 1
fns = glob.glob(f'/shares/zne.uzh/mrenke/ds-dnumrisk/derivatives/decoded_pdfs{stim}.volume.cv_vselect.denoise/sub-*/func/*_ses-1_mask-NPC_R_space-T1w_pars.tsv')
                # sub-40_ses-1_mask-NPC_R_space-T1w_pars.tsv  sub-40_ses-1_mask-NPC_R_space-T1w_r2s.tsv
reg = re.compile('.*/sub-(?P<subject>[0-9]+)_(ses-1_mask-NPC_R_space-T1w_pars.tsv)')

In [9]:
data = []
for fn in fns:
    try:
        data.append(reg.match(fn).groupdict())
        data[-1]['fn'] = fn
    except Exception as e:
        print(f'Problem with {fn}: {e}')

data = pd.DataFrame(data)
data['subject'] = [int(sub) for sub in data['subject']]
df = data.set_index(['subject'])
#df.sort_values(by='subject')

df_subjects_set = set(df.index)
missing_subjects = all_subjects_set - df_subjects_set
print(*missing_subjects, sep=',') # 


3,21


# missing:

* {3, 4, 10, 56, 60} --> problems with brain-mask from fmriprep?! --> just rerun it (deleting the workflow folders!!)
* 5.08.24  only sub-3 missing